In [1]:
using GLM
using TimeSeries, DataFrames
using XLSX
using StatsBase
using PrettyTables


In [2]:
data = DataFrame(XLSX.readtable("assets_data_03.24.xlsx", "Returns";infer_eltypes=true))
data = TimeArray(data,timestamp=:Date)



615×10 TimeArray{Any, 2, Date, Matrix{Any}} 1973-01-31 to 2024-03-31
┌────────────┬────────────┬─────────────┬──────────────┬──────────────┬─────────
│            │ ACWI_Net   │ USDPLN      │ ACWI_Net_PLN │ CPI_USA      │ CPI_PL ⋯
├────────────┼────────────┼─────────────┼──────────────┼──────────────┼─────────
│ 1973-01-31 │    missing │     missing │      missing │   0.00235344 │     mi ⋯
│ 1973-02-28 │    missing │     missing │      missing │   0.00704374 │     mi ⋯
│ 1973-03-31 │    missing │     missing │      missing │   0.00932044 │     mi ⋯
│ 1973-04-30 │    missing │     missing │      missing │   0.00692989 │     mi ⋯
│ 1973-05-31 │    missing │     missing │      missing │   0.00688219 │     mi ⋯
│ 1973-06-30 │    missing │     missing │      missing │   0.00683515 │     mi ⋯
│ 1973-07-31 │    missing │     missing │      missing │   0.00225755 │     mi ⋯
│ 1973-08-31 │    missing │     missing │      missing │    0.0180626 │     mi ⋯
│     ⋮      │     ⋮      │      ⋮      

In [3]:
colnames(data)

10-element Vector{Symbol}:
 :ACWI_Net
 :USDPLN
 :ACWI_Net_PLN
 :CPI_USA
 :CPI_PLN
 :TBSP
 :WIBOR3m
 :TenYPL
 :dWIBOR3m
 :dTenYPL

In [4]:
function returns_summarystats(data::TimeArray,t)
    names = colnames(data)
    returns = transpose(values(data))
    n_assets = size(returns)[1]

    stats = [ Dict(
        :mean => mean(returns[i,:]) * t, 
        :std => std(returns[i,:]) * t^0.5,
        :median => median(returns[i,:] * t),
        :skewness => skewness(returns[i,:]),
        :kurtosis => kurtosis(returns[i,:]),
        :autocor => autocor(returns[i,:],[1])[1],
        :p25th => percentile(returns[i,:],25) * t,
        :p75th => percentile(returns[i,:],75) * t,
        :min => minimum(returns[i,:]) * t,
        :max => maximum(returns[i,:]) * t,
        ) for i in 1:n_assets ]

    short_stats = pretty_table(stats, backend = Val(:html), row_labels = names)
    return short_stats
end



returns_summarystats (generic function with 1 method)

In [5]:
short = Float64.(from(data[:TBSP, :WIBOR3m, :TenYPL],Date(2007,01,01)))

207×3 TimeArray{Float64, 2, Date, Matrix{Float64}} 2007-01-31 to 2024-03-31
┌────────────┬─────────────┬────────────┬────────────┐
│            │ TBSP        │ WIBOR3m    │ TenYPL     │
├────────────┼─────────────┼────────────┼────────────┤
│ 2007-01-31 │     0.00626 │ 0.00349167 │     0.0043 │
│ 2007-02-28 │ 0.000149067 │ 0.00351667 │ 0.00435333 │
│ 2007-03-31 │  0.00510726 │ 0.00353333 │ 0.00434083 │
│ 2007-04-30 │ 0.000692007 │ 0.00366667 │ 0.00443917 │
│ 2007-05-31 │  0.00362559 │     0.0037 │ 0.00443917 │
│ 2007-06-30 │ -0.00847508 │   0.003925 │ 0.00470083 │
│ 2007-07-31 │  0.00471553 │ 0.00400833 │ 0.00466583 │
│ 2007-08-31 │ -0.00094856 │ 0.00419167 │ 0.00482417 │
│     ⋮      │      ⋮      │     ⋮      │     ⋮      │
│ 2023-09-30 │  0.00622425 │ 0.00480833 │ 0.00491583 │
│ 2023-10-31 │   0.0057372 │ 0.00471667 │ 0.00471167 │
│ 2023-11-30 │  0.00315821 │ 0.00485833 │   0.004585 │
│ 2023-12-31 │   0.0171423 │     0.0049 │  0.0043475 │
│ 2024-01-31 │   0.0045132 │ 0.00488333 │   

In [6]:
returns_summarystats(short,12)

,std,min,max,mean,skewness,p25th,autocor,kurtosis,p75th,median
TBSP,0.0428527,-0.502584,0.827558,0.0403455,0.342799,-0.0282004,0.0553435,5.80245,0.104502,0.0448318
WIBOR3m,0.00580118,0.0021,0.0751,0.0337908,0.271898,0.0172,0.988687,-1.03315,0.04915,0.0274
TenYPL,0.00476751,0.01149,0.08337,0.042673,-0.144031,0.02954,0.977263,-1.17253,0.057915,0.04218


In [7]:
varls = to(Float64.(from(data[:WIBOR3m, :TenYPL],Date(2003,01,01))),Date(2006,12,31))
returns_summarystats(varls,12)

,std,min,max,mean,skewness,p25th,autocor,kurtosis,p75th,median
WIBOR3m,0.0026271,0.0413,0.07,0.0532854,0.168329,0.04475,0.937251,-1.21056,0.05985,0.0542
TenYPL,0.00246801,0.04622,0.07497,0.0581775,0.642365,0.05172,0.912227,-0.865177,0.0661475,0.055605


In [8]:
varls

48×2 TimeArray{Float64, 2, Date, Matrix{Float64}} 2003-01-31 to 2006-12-31
┌────────────┬────────────┬────────────┐
│            │ WIBOR3m    │ TenYPL     │
├────────────┼────────────┼────────────┤
│ 2003-01-31 │ 0.00539167 │ 0.00477833 │
│ 2003-02-28 │   0.005225 │  0.0046275 │
│ 2003-03-31 │ 0.00498333 │   0.004565 │
│ 2003-04-30 │ 0.00474167 │ 0.00441583 │
│ 2003-05-31 │     0.0045 │ 0.00421583 │
│ 2003-06-30 │ 0.00443333 │  0.0042525 │
│ 2003-07-31 │ 0.00436667 │ 0.00472417 │
│ 2003-08-31 │ 0.00428333 │ 0.00471167 │
│     ⋮      │     ⋮      │     ⋮      │
│ 2006-06-30 │ 0.00350833 │  0.0047375 │
│ 2006-07-31 │ 0.00348333 │  0.0046675 │
│ 2006-08-31 │ 0.00349167 │ 0.00466167 │
│ 2006-09-30 │ 0.00351667 │ 0.00459167 │
│ 2006-10-31 │ 0.00350833 │  0.0044175 │
│ 2006-11-30 │     0.0035 │ 0.00431083 │
│ 2006-12-31 │     0.0035 │  0.0043525 │
└────────────┴────────────┴────────────┘
                         33 rows omitted

In [9]:
any(ismissing.(varls))

1×2 TimeArray{Bool, 2, Date, BitMatrix} 2006-12-31 to 2006-12-31
┌────────────┬─────────┬────────┐
│            │ WIBOR3m │ TenYPL │
├────────────┼─────────┼────────┤
│ 2006-12-31 │   false │  false │
└────────────┴─────────┴────────┘

In [10]:
varls[1:end-1]

47×2 TimeArray{Float64, 2, Date, Matrix{Float64}} 2003-01-31 to 2006-11-30
┌────────────┬────────────┬────────────┐
│            │ WIBOR3m    │ TenYPL     │
├────────────┼────────────┼────────────┤
│ 2003-01-31 │ 0.00539167 │ 0.00477833 │
│ 2003-02-28 │   0.005225 │  0.0046275 │
│ 2003-03-31 │ 0.00498333 │   0.004565 │
│ 2003-04-30 │ 0.00474167 │ 0.00441583 │
│ 2003-05-31 │     0.0045 │ 0.00421583 │
│ 2003-06-30 │ 0.00443333 │  0.0042525 │
│ 2003-07-31 │ 0.00436667 │ 0.00472417 │
│ 2003-08-31 │ 0.00428333 │ 0.00471167 │
│     ⋮      │     ⋮      │     ⋮      │
│ 2006-05-31 │ 0.00345833 │  0.0044425 │
│ 2006-06-30 │ 0.00350833 │  0.0047375 │
│ 2006-07-31 │ 0.00348333 │  0.0046675 │
│ 2006-08-31 │ 0.00349167 │ 0.00466167 │
│ 2006-09-30 │ 0.00351667 │ 0.00459167 │
│ 2006-10-31 │ 0.00350833 │  0.0044175 │
│ 2006-11-30 │     0.0035 │ 0.00431083 │
└────────────┴────────────┴────────────┘
                         32 rows omitted

In [11]:
diff_varls = varls[2:end] .- varls[1:end-1]

TypeError: TypeError: non-boolean (Missing) used in boolean context

In [12]:
ols = lm(@formula(TBSP ~ WIBOR3m + TenYPL), short[:TBSP, :WIBOR3m, :TenYPL])

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

TBSP ~ 1 + WIBOR3m + TenYPL

Coefficients:
──────────────────────────────────────────────────────────────────────────────
                   Coef.  Std. Error      t  Pr(>|t|)    Lower 95%   Upper 95%
──────────────────────────────────────────────────────────────────────────────
(Intercept)   0.00787004  0.00266261   2.96    0.0035   0.00262028   0.0131198
WIBOR3m       4.73141     1.18903      3.98    <1e-04   2.38705      7.07577
TenYPL       -5.01425     1.44683     -3.47    0.0006  -7.86691     -2.16159
──────────────────────────────────────────────────────────────────────────────

In [13]:
println(round(r2(ols),digits=4))


0.0726


In [14]:
res = residuals(ols)

207-element Vector{Float64}:
  0.003430716591018009
 -0.0025310752522496593
  0.002285585345042713
 -0.0022674570120775547
  0.0005084084603820815
 -0.011344760884167953
  0.0012760621939868766
 -0.004461530558817827
  0.0027820739511106307
  0.0011500434712708452
  ⋮
  0.005338244277345139
 -0.009133260284113345
  0.00025321597553191674
 -0.0008238605338106754
 -0.004708281211956773
  0.007887752722751981
 -0.004624845680767025
 -0.011182006298309195
 -0.00807717870245278

In [15]:
deviance(ols)

0.029236942219919678

In [16]:
(sum(res.^2)/(207-3))

0.00014331834421529255

In [17]:
predicted = predict(ols,varls)
display(mean(predicted)*12)
display(std(predicted)*12^0.5)

0.05483928570743096

0.010535360864781951

In [18]:
random_res = mean(rand(res,(10000,48)),dims=1)

1×48 Matrix{Float64}:
 -0.000105462  -9.55346e-5  2.84921e-5  …  0.0002976  -0.000121687

In [19]:
backfilled = predicted .+ random_res[1]
display(mean(backfilled)*12)
display(std(backfilled)*12^0.5)

0.05357374705727773

0.010535360864781953

In [20]:
backfilled = TimeArray(timestamp(varls),backfilled,[:TBSP])

48×1 TimeArray{Float64, 1, Date, Vector{Float64}} 2003-01-31 to 2006-12-31
┌────────────┬──────────────┐
│            │ TBSP         │
├────────────┼──────────────┤
│ 2003-01-31 │   0.00931502 │
│ 2003-02-28 │   0.00928277 │
│ 2003-03-31 │   0.00845274 │
│ 2003-04-30 │   0.00805727 │
│ 2003-05-31 │    0.0079167 │
│ 2003-06-30 │   0.00741741 │
│ 2003-07-31 │   0.00473693 │
│ 2003-08-31 │   0.00440532 │
│     ⋮      │      ⋮       │
│ 2006-06-30 │  0.000608945 │
│ 2006-07-31 │  0.000841657 │
│ 2006-08-31 │  0.000910335 │
│ 2006-09-30 │   0.00137962 │
│ 2006-10-31 │    0.0022135 │
│ 2006-11-30 │   0.00270893 │
│ 2006-12-31 │       0.0025 │
└────────────┴──────────────┘
              33 rows omitted

In [21]:
ls = merge(backfilled, varls)

48×3 TimeArray{Float64, 2, Date, Matrix{Float64}} 2003-01-31 to 2006-12-31
┌────────────┬──────────────┬────────────┬────────────┐
│            │ TBSP         │ WIBOR3m    │ TenYPL     │
├────────────┼──────────────┼────────────┼────────────┤
│ 2003-01-31 │   0.00931502 │ 0.00539167 │ 0.00477833 │
│ 2003-02-28 │   0.00928277 │   0.005225 │  0.0046275 │
│ 2003-03-31 │   0.00845274 │ 0.00498333 │   0.004565 │
│ 2003-04-30 │   0.00805727 │ 0.00474167 │ 0.00441583 │
│ 2003-05-31 │    0.0079167 │     0.0045 │ 0.00421583 │
│ 2003-06-30 │   0.00741741 │ 0.00443333 │  0.0042525 │
│ 2003-07-31 │   0.00473693 │ 0.00436667 │ 0.00472417 │
│ 2003-08-31 │   0.00440532 │ 0.00428333 │ 0.00471167 │
│     ⋮      │      ⋮       │     ⋮      │     ⋮      │
│ 2006-06-30 │  0.000608945 │ 0.00350833 │  0.0047375 │
│ 2006-07-31 │  0.000841657 │ 0.00348333 │  0.0046675 │
│ 2006-08-31 │  0.000910335 │ 0.00349167 │ 0.00466167 │
│ 2006-09-30 │   0.00137962 │ 0.00351667 │ 0.00459167 │
│ 2006-10-31 │    0.0022135 │

In [22]:
CSV.write("tbsp_backfilled.csv",ls)

UndefVarError: UndefVarError: `CSV` not defined

In [23]:
shortd = Float64.(from(data[:TBSP, :dWIBOR3m, :dTenYPL],Date(2007,01,01)))

207×3 TimeArray{Float64, 2, Date, Matrix{Float64}} 2007-01-31 to 2024-03-31
┌────────────┬─────────────┬──────────┬──────────┐
│            │ TBSP        │ dWIBOR3m │ dTenYPL  │
├────────────┼─────────────┼──────────┼──────────┤
│ 2007-01-31 │     0.00626 │  -0.0001 │ -0.00063 │
│ 2007-02-28 │ 0.000149067 │   0.0003 │  0.00064 │
│ 2007-03-31 │  0.00510726 │   0.0002 │ -0.00015 │
│ 2007-04-30 │ 0.000692007 │   0.0016 │  0.00118 │
│ 2007-05-31 │  0.00362559 │   0.0004 │      0.0 │
│ 2007-06-30 │ -0.00847508 │   0.0027 │  0.00314 │
│ 2007-07-31 │  0.00471553 │    0.001 │ -0.00042 │
│ 2007-08-31 │ -0.00094856 │   0.0022 │   0.0019 │
│     ⋮      │      ⋮      │    ⋮     │    ⋮     │
│ 2023-09-30 │  0.00622425 │  -0.0088 │  0.00329 │
│ 2023-10-31 │   0.0057372 │  -0.0011 │ -0.00245 │
│ 2023-11-30 │  0.00315821 │   0.0017 │ -0.00152 │
│ 2023-12-31 │   0.0171423 │   0.0005 │ -0.00285 │
│ 2024-01-31 │   0.0045132 │  -0.0002 │   9.0e-5 │
│ 2024-02-29 │ -0.00240331 │      0.0 │  0.00086 │
│ 2024

In [24]:
olsd = lm(@formula(TBSP ~ dWIBOR3m + dTenYPL), shortd)

StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Vector{Float64}}, GLM.DensePredChol{Float64, LinearAlgebra.CholeskyPivoted{Float64, Matrix{Float64}, Vector{Int64}}}}, Matrix{Float64}}

TBSP ~ 1 + dWIBOR3m + dTenYPL

Coefficients:
─────────────────────────────────────────────────────────────────────────────────
                   Coef.   Std. Error       t  Pr(>|t|)    Lower 95%    Upper 95%
─────────────────────────────────────────────────────────────────────────────────
(Intercept)   0.00346296  0.000230418   15.03    <1e-34   0.00300865   0.00391726
dWIBOR3m     -0.835608    0.103688      -8.06    <1e-13  -1.04005     -0.631171
dTenYPL      -3.26993     0.0739324    -44.23    <1e-99  -3.4157      -3.12416
─────────────────────────────────────────────────────────────────────────────────

In [25]:
r2(olsd)

0.9289694570840676

In [26]:
vardls = to(Float64.(from(data[:dWIBOR3m, :dTenYPL],Date(2003,02,01))),Date(2006,12,31))
returns_summarystats(vardls,12)

,std,min,max,mean,skewness,p25th,autocor,kurtosis,p75th,median
dWIBOR3m,0.00757375,-0.0648,0.0744,-0.00579574,0.936545,-0.021,0.479925,2.07982,0.0018,-0.0048
dTenYPL,0.0121559,-0.09552,0.12468,-0.00130468,0.437535,-0.02172,0.0740282,0.717512,0.02088,-0.00588


In [27]:
predictedd = predict(olsd,vardls)
display(mean(predictedd)*12)
display(std(predictedd)*12^0.5)

0.050664694874785296

0.04350609268871154

In [28]:
predictedd

47-element Vector{Union{Missing, Float64}}:
  0.011052751399214019
  0.0083386712729715
  0.011739400134342434
  0.013734058408800414
  0.0026926754821313343
 -0.01437636745667285
  0.004789056761768258
 -0.012414334697508274
 -0.03569240121221049
 -0.004166893857709727
  ⋮
 -0.0020232761327380775
 -0.002001451988527866
 -0.008613963929931015
  0.006460383834074508
  0.0036082932297641424
  0.005959018922689272
  0.010380676693786142
  0.007732032099833799
  0.0018279930377693046

In [29]:
res = residuals(olsd)

207-element Vector{Float64}:
  0.0006534234385229907
 -0.0009704533168588005
  0.001320935508712469
  0.002424540861044975
  0.0004968710023352872
  0.00058569170309156
  0.0007148075577032217
  0.003639688845070375
  0.0011546612105229155
 -0.0021040380642500314
  ⋮
  0.0013210354772392463
  0.002368484560918753
  0.006166015574676678
 -0.006656255524975235
 -0.0038545150589394655
  0.004777806539335044
  0.0011774175418032074
 -0.003054128666302041
  0.0009070292338371217

In [30]:
mean(rand(res,(10000,48)),dims=1)

1×48 Matrix{Float64}:
 3.62248e-5  5.76011e-5  3.22075e-6  …  -4.13751e-5  3.82863e-6  9.24834e-6

In [31]:
backfilledd = predictedd .+ mean(rand(res,(10000,48)),dims=1)
display(mean(backfilledd)*12)
display(std(backfilledd)*12^0.5)
backfilledd = TimeArray(timestamp(vardls),backfilledd[:,1],[:TBSP])

0.050686322446146556

0.04305046732673662

47×1 TimeArray{Float64, 1, Date, Vector{Float64}} 2003-02-28 to 2006-12-31
┌────────────┬─────────────┐
│            │ TBSP        │
├────────────┼─────────────┤
│ 2003-02-28 │   0.0110263 │
│ 2003-03-31 │  0.00831221 │
│ 2003-04-30 │   0.0117129 │
│ 2003-05-31 │   0.0137076 │
│ 2003-06-30 │  0.00266621 │
│ 2003-07-31 │  -0.0144028 │
│ 2003-08-31 │   0.0047626 │
│ 2003-09-30 │  -0.0124408 │
│     ⋮      │      ⋮      │
│ 2006-06-30 │ -0.00864043 │
│ 2006-07-31 │  0.00643392 │
│ 2006-08-31 │  0.00358183 │
│ 2006-09-30 │  0.00593256 │
│ 2006-10-31 │   0.0103542 │
│ 2006-11-30 │  0.00770557 │
│ 2006-12-31 │  0.00180153 │
└────────────┴─────────────┘
             32 rows omitted

In [32]:
returns_summarystats(backfilledd,12)

,std,min,max,mean,skewness,p25th,autocor,kurtosis,p75th,median
TBSP,0.0435061,-0.428626,0.376645,0.0503472,-0.564735,-0.0244659,0.11399,1.09193,0.128283,0.0584162


In [33]:
CSV.write("tbsp_backfilledd.csv",backfilled)

UndefVarError: UndefVarError: `CSV` not defined

In [34]:
yearly = cumsum()

MethodError: MethodError: no method matching cumsum()

Closest candidates are:
  cumsum(!Matched::FillArrays.AbstractOnes{Bool, 1})
   @ FillArrays C:\Users\matsz\.julia\packages\FillArrays\eOEVm\src\FillArrays.jl:587
  cumsum(!Matched::FillArrays.AbstractOnes{T, 1}) where T<:Integer
   @ FillArrays C:\Users\matsz\.julia\packages\FillArrays\eOEVm\src\FillArrays.jl:586
  cumsum(!Matched::FillArrays.AbstractZeros{Bool, 1})
   @ FillArrays C:\Users\matsz\.julia\packages\FillArrays\eOEVm\src\FillArrays.jl:585
  ...
